In [8]:
#!python3
'''
Maj Smith addition.  This is a greedy algo that just meets the requirements without any special consideration other than the min constraints.
Goal:  Pass the checker function.
'''
from DataLoader import *
from CheckSchedule import checkSchedule
from satUtils import satisfiable, satisfiableMember
import copy
from pprint import pprint
import random
import sys
sys.setrecursionlimit(10000)

def times_overlap(x1, x2, y1, y2) -> bool:
    return max(x1, y1) <= min(x2, y2)

def already_assigned_to_a_flight(member: int, event_dates: tuple) -> bool:
    try:
        assigned = schedule["Staff"][member]
        proposed_event_start = event_dates[0]
        proposed_event_end = event_dates[1]
    except:
        return False
    for flight in assigned:
        booked_start = events[flight]['StartDay']
        booked_end = events[flight]['EndDay']
        if times_overlap(proposed_event_start, proposed_event_end, booked_start, booked_end):
            return True
    return False

# assumption: unavailability begins at start day of event
def staff_on_leave(event: dict,
                       member: int,
                       member_details: dict) -> bool:
    # check if on leave
    if not member_details['Leave']:
        return False
    else:
        for dates in member_details['Leave']:
            leave_start = dates[0]
            leave_end = dates[-1]
            # if no overlap:
            if times_overlap(leave_start, leave_end, event['StartDay'], event['EndDay']):
                return True
    return False

def datenum_to_month(datenum_start: int) -> str:
    monthNames = ["Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    july = (60, 90)
    august = (91, 121)
    september = (122, 151)
    october = (152, 182)
    november = (183, 212)
    december = (213, 243)
    months = [july, august, september, october, november, december]
    monthDateNames = list(zip(months, monthNames))
    for days, month in monthDateNames:
        # in date range?
        if datenum_start >= days[0] and datenum_start <= days[-1]:
            return month
    print(datenum_start)
    print('Could not convert to month')
    

def check_aircrew_credentials(event: dict, available_staff: dict) -> list:
    crew_min = event['CrewMin']
    crew_max = event['CrewMax']
    crew_met = 0
    chosen_crew = []
    for req in event['CrewRequirements']:
        if req == '':
            # TODO: future change here! Optimize!
            while True:
                print('training event')
                if crew_met >= crew_min:
                    break
                lucky_dude = random.choice(list(available_staff.items()))
                lucky_dude_id = lucky_dude[0]
                lucky_dude = {lucky_dude_id: lucky_dude[1]}
                # I think this exception has to do with new pilots showing up, and not having a record for prior months.
                month = datenum_to_month(event['StartDay'])
                try:
                    lucky_dude_qual = lucky_dude[lucky_dude_id]['Quals'][month]
                except KeyError:
                    lucky_dude_qual = lucky_dude[lucky_dude_id]['Quals']['Aug']
                if lucky_dude_qual == None:
                    pass
                # TODO: make sure these are the least qualified people:
                if lucky_dude not in chosen_crew and lucky_dude_qual in ['', None, 'none', 'FPNC', "FPCC","FPKC"]: # ,"FPQC","FPLC"
                    chosen_crew.append(lucky_dude)
                    crew_met += 1
                    break
        else:
            # TODO: future change here!  Optimize!
            counter = 0
            big_counter = 0
            while True:
                print('live event')
                if crew_met >= crew_min:
                    break
                lucky_dude = random.choice(list(available_staff.items()))
                lucky_dude_id = lucky_dude[0]
                lucky_dude = {lucky_dude_id: lucky_dude[1]}
                month = datenum_to_month(event['StartDay'])
                # I think this exception has to do with new pilots showing up, and not having a record for prior months.
                try:
                    lucky_dude_qual = lucky_dude[lucky_dude_id]['Quals'][month]
                except KeyError:
                    lucky_dude_qual = lucky_dude[lucky_dude_id]['Quals']['Aug']
                if lucky_dude_qual == None:
                    continue
                if lucky_dude not in chosen_crew:
                    if satisfiableMember(req, lucky_dude_qual):
                        chosen_crew.append(lucky_dude)
                        crew_met += 1
                counter += 1
                big_counter += 1
                if counter > 40:
                    chosen_crew = []
                    counter = 0
                    crew_met = 0
                if big_counter > 1000:
                    return []
    assert(crew_met >= crew_min and crew_met <= crew_max)
    return chosen_crew

schedule = {
    "Events": {},
    "Staff": {},
}
events = loadEventData()
staff = loadPilotData()

original_events = copy.deepcopy(events) # for record, if needed

event_details = events['f1']

staff_not_on_leave = {member: details for member, details in staff.items()
    if not staff_on_leave(event_details, member, details)}

staff_available = {member: details for member, details in staff_not_on_leave.items()
    if not already_assigned_to_a_flight(member, (event_details['StartDay'], event_details['EndDay']))}

# find crew with the credentials needed and meet min crew needs
chosen_crew = check_aircrew_credentials(event_details, staff_available)
chosen_crew


# pprint(schedule)
# checkSchedule(schedule, verbose=True)
    


live event
live event
live event
live event
live event
live event
live event


[{10005: {'Leave': [[95.0, 101.0], [149.0, 150.0]],
   'Quals': {'Aug': 'IP A2',
    'Sep': 'IPA2',
    'Oct': 'IP A2',
    'Nov': 'IPA2',
    'Dec': 'IP A2'}}},
 {10014: {'Leave': [[72.0, 76.0],
    [81.0, 82.0],
    [94.0, 96.0],
    [116.0, 133.0],
    [237.0, 241.0],
    [154.0, 245.0]],
   'Quals': {'Aug': 'IP BZ',
    'Sep': 'IP BZ',
    'Oct': 'IP BZ',
    'Nov': 'IP BZ',
    'Dec': 'IP BJ'}}},
 {10099: {'Leave': [[88.0, 91.0], [154.0, 245.0]],
   'Quals': {'Aug': 'IP A1',
    'Sep': 'IP A1',
    'Oct': 'IP A1',
    'Nov': 'IP A1',
    'Dec': 'IP A1'}}}]

In [9]:
events['f1']

'3.8.3 (default, May 19 2020, 13:54:14) \n[Clang 10.0.0 ]'